In [82]:
import pandas as pd 
from pulp import *

In [83]:
cash = LpProblem("Cash",LpMinimize)
month = [1,2,3,4,5,6]
receive = {1:1.5,
           2:1,
           3:1.4,
           4:2.3,
           5:2,
           6:1}
bills = {1:1.8,
         2:1.6,
         3:2.2,
         4:1.2,
         5:0.8,
         6:1.2}
min_allow = 0.25
initial = 0.4

#Make the dictionaries:
delay = LpVariable.dicts("d",month,lowBound=0,cat="Continuous")
min_balance = LpVariable.dicts("m",month,lowBound=min_allow,cat="Continuous")
borrow = LpVariable.dicts("b",month,lowBound=0,cat="Continuous")
shortloan = LpVariable("s",lowBound=0,cat="Continuous")

In [84]:
#The total cost is as follows, as well as the objective. 
cash += -initial*0.005 + shortloan*0.06 +\
        lpSum([borrow[i]*0.015 +\
        delay[i]*(1/0.98-1) -\
        min_balance[i]*0.005 for i in range(1,6)])
#Set up the constraints :
#1st month :
cash += initial*1.005+shortloan+receive[1]+delay[1]+borrow[1]\
        == bills[1]+min_balance[1]+0.01*shortloan
#2nd - 5th months:
for i in range(2,6):
    cash += receive[i]+min_balance[i-1]*1.005+delay[i]+borrow[i]\
            == bills[i]+min_balance[i]+0.01*shortloan+\
            delay[i-1]*(1/0.98)+1.015*borrow[i-1]
#6th month
cash += receive[6]+min_balance[5]*1.005\
        == bills[6]+1.015*borrow[5]+delay[5]*(1/0.98)+\
           1.01*shortloan+min_balance[6]
#Can only delay part of the bills:
for i in range(1,7):
    cash += delay[i]<=bills[i]
#Can only borrow 0.75 of the entire receive
for i in range(1,7):
    cash += borrow[i]<=receive[i]*0.75

In [85]:
cash.writeLP("problem 1.lp")
cash.solve()
print("total cost is", value(cash.objective))
for v in cash.variables():
    print(v,v.value())

total cost is 0.03349266416938776
b_1 0.0
b_2 0.60024495
b_3 1.05
b_4 0.33282513
b_5 0.0
b_6 0.0
d_1 0.0
d_2 0.0
d_3 0.35949357
d_4 0.0
d_5 0.0
d_6 0.0
m_1 0.25
m_2 0.25
m_3 0.25
m_4 0.25
m_5 1.1119375
m_6 0.76650734
s 0.14949495
